In [ ]:
## project 2

### This project is basically about apply various ML models and get the best model out of all applied. The data set is very small with 400 rows which seems like a sampple data. Initially we are applying EDA no feature engineering is required here since there are only 3 columns.

### After we are done with EDA we are applying various ML models such as Logistic, Knn,DT and R-forest models. For this process `we are performing grid search and hyper parameter tunning to improve the performance of each models.



In [ ]:
#3 import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
## import dataset
df= pd.read_csv("Social_Network_Ads.csv")
df.head()

In [ ]:
## check the dimension
df.shape

In [ ]:
#3 drop the unwanted columns
df.drop(['User ID', "Gender"], axis=1, inplace=True)

In [ ]:
#3 display the samples
df.sample(10)

In [ ]:
df.Purchased.value_counts()

### The dataset seems to be imbalanced as not purchased data are about double of the purcahsed ones.

In [ ]:
## display the info about dtypea nd null values
df.info()

In [ ]:
#3 display the statistical values
df.describe().T

In [ ]:
## plot a pairplot to see the correlation between features
sns.pairplot(df, hue='Purchased')

In [ ]:
## joit plot for correlation
sns.jointplot(x='Age',y='EstimatedSalary',data=df)

In [ ]:
## start  modeling
X= df.drop('Purchased', axis=1)
y= df['Purchased']

In [ ]:
## import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:


# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Scale the features
scaler = StandardScaler()

X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

In [ ]:
resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Purchased'])], axis=1)
resampled_df.Purchased.value_counts()

In [ ]:
# Logistic Regression
from sklearn.model_selection import GridSearchCV

# Define hyperparameters grid for Logistic Regression
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga'],  # Algorithm to use in the optimization problem
    'max_iter': [100, 200, 300],  # Maximum number of iterations for the solvers
    'class_weight': ['balanced', None]  # Weights associated with classes
}

# Initialize Logistic Regression model
logistic_model = LogisticRegression()

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(estimator=logistic_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_resampled_scaled, y_resampled)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Use the best parameters to train the Logistic Regression model
best_logistic_model = LogisticRegression(**best_params)
best_logistic_model.fit(X_resampled_scaled, y_resampled)
best_logistic_y_pred = best_logistic_model.predict(X_test_scaled)

# Print the classification report for the best Logistic Regression model
print("Logistic Regression Classification Report (Best Model):")
print(classification_report(y_test, best_logistic_y_pred))

In [ ]:
## 1. Knn
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Define the KNN model
knn_model = KNeighborsClassifier()

# Perform GridSearchCV
knn_grid_search = GridSearchCV(estimator=knn_model, param_grid=knn_param_grid, cv=5, scoring='accuracy')
knn_grid_search.fit(X_resampled_scaled, y_resampled)

# Get the best parameters and score
knn_best_params = knn_grid_search.best_params_
knn_best_score = knn_grid_search.best_score_

# Print best parameters and score
print("KNN Best Parameters:", knn_best_params)
print("KNN Best Score:", knn_best_score)

# Create KNN model with best parameters
knn_model = KNeighborsClassifier(**knn_best_params)

# Fit the model to the resampled data
knn_model.fit(X_resampled_scaled, y_resampled)

# Make predictions on the test set
knn_y_pred = knn_model.predict(X_test_scaled)

# Print classification report
print("KNN Classification Report:")
print(classification_report(y_test, knn_y_pred))

In [ ]:

# Define the Decision Tree parameter grid
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the Decision Tree model
dt_model = DecisionTreeClassifier()

# Perform GridSearchCV
dt_grid_search = GridSearchCV(estimator=dt_model, param_grid=dt_param_grid, cv=5, scoring='accuracy')
dt_grid_search.fit(X_resampled_scaled, y_resampled)

# Get the best parameters and score
dt_best_params = dt_grid_search.best_params_
dt_best_score = dt_grid_search.best_score_

# Print best parameters and score
print("Decision Tree Best Parameters:", dt_best_params)
print("Decision Tree Best Score:", dt_best_score)

# Create Decision Tree model with best parameters
dt_model = DecisionTreeClassifier(**dt_best_params)

# Fit the model to the resampled data
dt_model.fit(X_resampled_scaled, y_resampled)

# Make predictions on the test set
dt_y_pred = dt_model.predict(X_test_scaled)

# Print classification report
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_y_pred))

In [ ]:
# Define the Random Forest parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Define the Random Forest model
rf_model = RandomForestClassifier()

# Perform GridSearchCV
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(X_resampled_scaled, y_resampled)

# Get the best parameters and score
rf_best_params = rf_grid_search.best_params_
rf_best_score = rf_grid_search.best_score_

# Print best parameters and score
print("Random Forest Best Parameters:", rf_best_params)
print("Random Forest Best Score:", rf_best_score)

# Create Random Forest model with best parameters
rf_model = RandomForestClassifier(**rf_best_params)

# Fit the model to the resampled data
rf_model.fit(X_resampled_scaled, y_resampled)

# Make predictions on the test set
rf_y_pred = rf_model.predict(X_test_scaled)

# Print classification report
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_y_pred))


In [ ]:
from sklearn.metrics import accuracy_score
log_accuracy = accuracy_score(y_test, best_logistic_y_pred)
Knn_accuracy = accuracy_score(y_test, knn_y_pred)
dtree_accuracy = accuracy_score(y_test, dt_y_pred)
rfc_accuracy = accuracy_score(y_test, rf_y_pred)

from sklearn.metrics import accuracy_score, classification_report
log_accuracy = accuracy_score(y_test, logistic_y_pred)
Knn_accuracy = accuracy_score(y_test, knn_y_pred)
dtree_accuracy = accuracy_score(y_test, dt_y_pred)
rfc_accuracy = accuracy_score(y_test, rf_y_pred)

In [ ]:
#3 create a df with above values
model_names = ['Logistic Regression', 'K-Nearest Neighbors',  'Decision Tree', 'Random Forest']
accuracy_scores = [log_accuracy, Knn_accuracy, dtree_accuracy, rfc_accuracy]

# Initialize DataFrame
accuracy_df = pd.DataFrame({'Model': model_names, 'Accuracy': accuracy_scores})

# Highlight the row with the best accuracy score
accuracy_df_style = accuracy_df.style.highlight_max(subset=['Accuracy'], color='lightgreen')

# Print the DataFrame
accuracy_df_style



### 4. Evaluate the model accuracy and suggest the best model out of all 5 models to be applied in the dataset.
### Answer: Among the four models used  the best models seems to be Knn and Random Forest Model with best accuracy of 93% with similar precision and recall values.

### ### 5. interpret the classification report for the dataset and explain why the model is best fit.

In [ ]:
print(classification_report(y_test, knn_y_pred))

In [ ]:
print(classification_report(y_test, rf_y_pred))

### Both knn and random forest models demonstrate nuanced performance across both classes. For class 0 (not purchased), there is high precision, suggesting that when the model predicts an instance as not purchased, they are  correct about 98% of the time. However, the recall for class 0 is slightly lower, indicating that while the models are accurate in labeling instances as not purchased, they might miss some actual instances of not being purchased. On the other hand, for class 1 (purchased), there is slightly lower precision but notably high recall. This indicates that while the models may occasionally mislabel an instance as purchased, they are effective at capturing most of the actual purchased instances. This quality is particularly valuable in scenarios like targeted marketing or fraud detection, where correctly identifying positive cases is crucial.
### Overall, with an accuracy of 93%, both models demonstrate strong predictive capabilities. Also, it's important to acknowledge the trade-offs between precision and recall. The high recall for class 1 suggests the model is adept at identifying purchased instances, but the slightly lower precision means there could be some false positives.

## The part below this line is an extra job which is not a part of project

In [ ]:
## lets apply pycaret library as apply some more models
!pip install pycaret


In [ ]:
from pycaret.classification import *
s = setup(df, target = 'Purchased', session_id = 123)




In [ ]:
best_model = compare_models()
tuned_model = tune_model(best_model)
evaluate_model(tuned_model)


